In [1]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: transformers in c:\users\uday kharatmol\appdata\local\programs\python\python39\lib\site-packages (4.51.1)



You should consider upgrading via the 'C:\Users\UDAY KHARATMOL\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
# Install required libraries
!pip install transformers torch datasets accelerate rouge_score

You should consider upgrading via the 'C:\Users\UDAY KHARATMOL\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
# In Jupyter Notebook
!pip install evaluate

In [ ]:
# Install all required libraries
!pip install transformers torch datasets accelerate evaluate rouge_score tiktoken

In [ ]:
# Import libraries
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline
import evaluate

# Enforce GPU usage - raise error if no GPU is available
if not torch.cuda.is_available():
    raise RuntimeError("No GPU available. Please ensure you have an NVIDIA GPU with CUDA installed and PyTorch with GPU support. RTX 3060 should work - check drivers and PyTorch installation.")

In [ ]:
# Set device to GPU only
device = torch.device("cuda")
print(f"Using GPU: {torch.cuda.get_device_name(0)}")

# Load the xlsum dataset for Hindi
dataset = load_dataset("csebuetnlp/xlsum", "hindi")

# Split into train and validation sets
train_dataset = dataset["train"]
val_dataset = dataset["validation"]

# Check a sample
print("Sample from training set:")
print("Article:", train_dataset[0]["text"])
print("Summary:", train_dataset[0]["summary"])

In [ ]:
# Load base model and tokenizer
model_name = "ai4bharat/IndicBART"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move model to GPU (no CPU fallback)
model.to(device)

# Preprocess the dataset
def preprocess_function(examples):
    inputs = [doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length", return_tensors="pt").to(device)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length", return_tensors="pt").to(device)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
# Apply preprocessing
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=["text", "summary", "id"])
tokenized_val = val_dataset.map(preprocess_function, batched=True, remove_columns=["text", "summary", "id"])

# Define training arguments (removed 'device' parameter)
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust based on RTX 3060 memory
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=100,
    num_train_epochs=100,
    predict_with_generate=True,
    push_to_hub=False,
    fp16=True,  # Enable mixed precision for faster GPU training
)

# Load ROUGE metric using evaluate
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(pred):
    predictions = pred.predictions
    labels = pred.label_ids

    # Move predictions and labels to GPU if not already (assuming device is set)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    if predictions.device != device:
        predictions = torch.tensor(predictions).to(device)
    if labels.device != device:
        labels = torch.tensor(labels).to(device)

    # Move back to CPU for decoding
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # New format: ROUGE scores are directly floats in a dictionary
    return {k: v * 100 for k, v in result.items()}  # Multiply by 100 to get percentages

In [14]:
# Initialize trainer with accelerator for GPU
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model (strictly on GPU)
trainer.train()

C:\Users\UDAY KHARATMOL\AppData\Local\Temp\ipykernel_15132\4292471213.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.000000,nan,60.448392,0.021005,60.092717,60.098867
2,0.000000,nan,60.448392,0.021005,60.092717,60.098867
3,0.000000,nan,60.448392,0.021005,60.092717,60.098867


TrainOutput(global_step=26544, training_loss=0.0, metrics={'train_runtime': 18552.0063, 'train_samples_per_second': 11.445, 'train_steps_per_second': 1.431, 'total_flos': 2.3008300963253453e+17, 'train_loss': 0.0, 'epoch': 3.0})

In [15]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

print("Training completed and model saved!")

Training completed and model saved!


In [24]:
# Enable CUDA debugging
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Import libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Enforce GPU usage
if not torch.cuda.is_available():
    raise RuntimeError("No GPU available. Please ensure CUDA and PyTorch GPU support are installed.")

device = torch.device("cuda")
print(f"Using GPU: {torch.cuda.get_device_name(0)}")

Using GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [25]:
# Step 1: Load tokenizer only
model_path = "./fine_tuned_model"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    print("Tokenizer loaded successfully.")
    print(f"Vocab size: {tokenizer.vocab_size}")
    print(f"Pad token ID: {tokenizer.pad_token_id}")
    print(f"EOS token ID: {tokenizer.eos_token_id}")
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    raise

Tokenizer loaded successfully.
Vocab size: 64000
Pad token ID: 0
EOS token ID: 3


In [26]:
# Step 2: Load model only
try:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

Model loaded successfully.


In [3]:
# Import libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

# Set device to CPU
device = torch.device("cpu")
print("Using CPU for testing.")

# Load the fine-tuned model and tokenizer
model_path = "./fine_tuned_model"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    print("Model and tokenizer loaded successfully.")
    print(f"Vocab size: {tokenizer.vocab_size}")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    raise

# Move model to CPU and set to evaluation mode
model.to(device)
model.eval()

# Ensure pad_token_id is set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    model.config.pad_token_id = tokenizer.pad_token_id
    print("Set pad_token_id to:", tokenizer.pad_token_id)

# Test 1: Minimal manual inference with short input
test_input = "भारत एक देश है।"
print("\nTest 1: Minimal manual inference with short text:", test_input)

try:
    inputs = tokenizer(
        test_input,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=50,
            min_length=10,
            do_sample=False
        )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Original Text:", test_input)
    print("Generated Summary:", summary)
except Exception as e:
    print(f"Manual inference failed with error: {e}")
    raise

# Test 2: Manual inference with dataset sample (avoiding pipeline)
print("\nTest 2: Manual inference with dataset sample")
try:
    dataset = load_dataset("csebuetnlp/xlsum", "hindi")
    train_dataset = dataset["train"]
    sample_article = train_dataset[0]["text"]
    print("Dataset sample text length:", len(sample_article))
    print("Dataset sample (first 200 chars):", sample_article[:200] + "...")
    
    # Tokenize
    inputs = tokenizer(
        sample_article,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    print("Tokenization successful.")
    print("Input IDs shape:", inputs["input_ids"].shape)
    
    # Generate summary manually
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=50,
            min_length=25,
            do_sample=False
        )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Original Article:", sample_article[:200] + "...")
    print("Generated Summary:", summary)
except Exception as e:
    print(f"Test 2 failed with error: {e}")
    raise

Using CPU for testing.
Model and tokenizer loaded successfully.
Vocab size: 64000

Test 1: Minimal manual inference with short text: भारत एक देश है।
Original Text: भारत एक देश है।
Generated Summary: मालूम भारत एक देश है।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।

Test 2: Manual inference with dataset sample
Dataset sample text length: 9593
Dataset sample (first 200 chars): नेपाल में इस दौरे की भरपूर तैयारियां चल रही हैं. इसकी ख़ास वजह भी है क्योंकि 23 साल बाद कोई चीनी राष्ट्रपति नेपाल पहुंच रहा है. नेपाल के प्रधानमंत्री केपी शर्मा ओली के विदेश मामलों के सलाहकार डॉ. राजन...
Tokenization successful.
Input IDs shape: torch.Size([1, 512])
Original Article: नेपाल में इस दौरे की भरपूर तैयारियां चल रही हैं. इसकी ख़ास वजह भी है क्योंकि 23 साल बाद कोई चीनी राष्ट्रपति नेपाल पहुंच रहा है. नेपाल के प्रधानमंत्री केपी शर्मा ओली के विदेश मामलों के सलाहकार डॉ. राजन...
Generated Summary: ग़ नेपाल में इस दौरे की भरपूर तैयारियां चल रही हैं. इसकी ख़ास वजह भी है क्योंकि 23 साल बाद कोई चीनी राष्ट्रपति नेपाल

In [4]:
# Import libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset

# Set device to CPU
device = torch.device("cpu")
print("Using CPU for testing.")

# Load the fine-tuned model and tokenizer
model_path = "./fine_tuned_model"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    print("Model and tokenizer loaded successfully.")
    print(f"Vocab size: {tokenizer.vocab_size}")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    raise

# Move model to CPU and set to evaluation mode
model.to(device)
model.eval()

# Ensure pad_token_id is set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    model.config.pad_token_id = tokenizer.pad_token_id
    print("Set pad_token_id to:", tokenizer.pad_token_id)

# Test 1: Minimal manual inference with short input
test_input = "भारत एक देश है।"
print("\nTest 1: Minimal manual inference with short text:", test_input)

try:
    inputs = tokenizer(
        test_input,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=50,
            min_length=10,
            do_sample=False
        )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Original Text:", test_input)
    print("Generated Summary:", summary)
except Exception as e:
    print(f"Test 1 failed with error: {e}")
    raise

# Test 2: Manual inference with dataset sample
print("\nTest 2: Manual inference with dataset sample")
try:
    dataset = load_dataset("csebuetnlp/xlsum", "hindi")
    train_dataset = dataset["train"]
    sample_article = train_dataset[0]["text"]
    print("Dataset sample (first 200 chars):", sample_article[:200] + "...")
    
    inputs = tokenizer(
        sample_article,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=50,
            min_length=25,
            do_sample=False
        )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Original Article:", sample_article[:200] + "...")
    print("Generated Summary:", summary)
except Exception as e:
    print(f"Test 2 failed with error: {e}")
    raise

# Test 3: User input for custom text
print("\nTest 3: Summarization with user input")
while True:
    try:
        custom_text = input("Enter Hindi text to summarize (or type 'exit' to stop): ")
        if custom_text.lower() == "exit":
            break
        if not custom_text.strip():
            print("Error: Input cannot be empty. Try again.")
            continue
        
        print("Processing input:", custom_text)
        inputs = tokenizer(
            custom_text,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=50,
                min_length=25,
                do_sample=False
            )
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("Generated Summary:", summary)
    except Exception as e:
        print(f"Test 3 failed with error: {e}")

Using CPU for testing.
Model and tokenizer loaded successfully.
Vocab size: 64000

Test 1: Minimal manual inference with short text: भारत एक देश है।
Original Text: भारत एक देश है।
Generated Summary: मालूम भारत एक देश है।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।

Test 2: Manual inference with dataset sample
Dataset sample (first 200 chars): नेपाल में इस दौरे की भरपूर तैयारियां चल रही हैं. इसकी ख़ास वजह भी है क्योंकि 23 साल बाद कोई चीनी राष्ट्रपति नेपाल पहुंच रहा है. नेपाल के प्रधानमंत्री केपी शर्मा ओली के विदेश मामलों के सलाहकार डॉ. राजन...
Original Article: नेपाल में इस दौरे की भरपूर तैयारियां चल रही हैं. इसकी ख़ास वजह भी है क्योंकि 23 साल बाद कोई चीनी राष्ट्रपति नेपाल पहुंच रहा है. नेपाल के प्रधानमंत्री केपी शर्मा ओली के विदेश मामलों के सलाहकार डॉ. राजन...
Generated Summary: ग़ नेपाल में इस दौरे की भरपूर तैयारियां चल रही हैं. इसकी ख़ास वजह भी है क्योंकि 23 साल बाद कोई चीनी राष्ट्रपति नेपाल पहुंच रहा है. नेपाल के प्रधानमंत्री केपी शर्मा ओली के विदेश मामलों के सलाहकार डॉ. राजन भट्टाराई

Enter Hindi text to summarize (or type 'exit' to stop):  दिल क्या करे जब किसी से किसी को प्यार हो जाए जाने कहाँ कब किसी को किसी से प्यार हो जाए ऊँची-ऊँची दीवारों सी इस दुनिया की रस्में न कुछ तेरे बस में जुली, न कुछ मेरे बस में


Processing input: दिल क्या करे जब किसी से किसी को प्यार हो जाए जाने कहाँ कब किसी को किसी से प्यार हो जाए ऊँची-ऊँची दीवारों सी इस दुनिया की रस्में न कुछ तेरे बस में जुली, न कुछ मेरे बस में
Generated Summary: दिल क्या करे जब किसी से किसी को प्यार हो जाए जाने कहाँ कब किसी को किसी से प्यार हो जाए ऊँची-ऊँची दीवारों सी इस दुनिया की रस्में न कुछ तेरे बस में जु तेरे बस में जु


Enter Hindi text to summarize (or type 'exit' to stop):  आ मैं तेरी याद में सब को भुला दूँ दुनिया को तेरी तस्वीर बना दूँ मेरा बस चले तो दिल चीर के दिखा दूँ हो, दौड़ रहा है साथ लहू के प्यार तेरे नस-नस में न कुछ तेरे बस में जुली, न कुछ मेरे बस में दिल क्या करे ...


Processing input: आ मैं तेरी याद में सब को भुला दूँ दुनिया को तेरी तस्वीर बना दूँ मेरा बस चले तो दिल चीर के दिखा दूँ हो, दौड़ रहा है साथ लहू के प्यार तेरे नस-नस में न कुछ तेरे बस में जुली, न कुछ मेरे बस में दिल क्या करे ...
Generated Summary: ख़ तेरी तेरी याद में सब को भुला दूँ दुनिया को तेरी तस्वीर बना दूँ मेरा बस चले तो दिल चीर के दिखा दूँ हो, दौड़ रहा है साथ लहू के प्यार तेरे नस-नस में न कुछ


Enter Hindi text to summarize (or type 'exit' to stop):  exit
